# Demonstration 
This example demonstrate the usage of the `pyiron_lammps` LAMMPS parser for simple interatomic potential calculations. Starting with the import of the required packages. In addition to the `pyiron_lammps` package and the atomic simulation environment `ase`, the `os` package and the `subprocess` package from the Python standard library are imported to create the working directory and execute the LAMMPS executable.

In [1]:
import os
import subprocess

from ase.build import bulk
from pyiron_lammps import parse_lammps_output_files, write_lammps_structure

As a first step the working directory is created, in this example it is named `demo`: 

In [2]:
working_directory = "demo"
os.makedirs(working_directory, exist_ok=True)

In addition to the atomistic structure, LAMMPS also requires an input script. These input scripts can vary in complexity, as they can include for loops and other logical structures. For simplicity, the example here is based on just a single `run` command, resulting in a static evaluation of the atomistic structure. For the example the [NiAlH_jea.eam.alloy](https://github.com/lammps/lammps/blob/develop/potentials/NiAlH_jea.eam.alloy) potential is used which is available as part of the LAMMPS repository. Furthermore, the `dump` commands and `thermo` commands for `pyiron_lammps` are already specified in this input script. 

In [3]:
lammps_input_script = """\
units metal
dimension 3
boundary p p p
atom_style atomic

read_data lammps.data

pair_style eam/alloy
pair_coeff * * ../binder/NiAlH_jea.eam.alloy Ni Al H

dump 1 all custom 100 dump.out id type xsu ysu zsu fx fy fz vx vy vz
dump_modify 1 sort id format line "%d %d %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g"
thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
thermo 100

run 0
"""

with open(os.path.join(working_directory, "lmp.in"), "w") as f:
    f.writelines(lammps_input_script)

After the input script is written the next step is to create the `lammps.data` file which contains the atomistic structure. Here a cubic bulk Aluminium structure is written to the input file. 

In [4]:
structure = bulk("Al", cubic=True)
potential_elements = ["Ni", "Al", "H"]

write_lammps_structure(
    structure=structure,
    potential_elements=potential_elements,
    units="metal",
    file_name="lammps.data",
    working_directory=working_directory,
)

The structure parameter refers to the `ase.atoms.Atoms` structure and the `potential_elements` refers to the list of elements implemented in the specific interatomic potential. For example the `NiAlH_jea.eam.alloy` potential implements the elements Ni, Al and H, so when writing a structure for a simulation with this potential the `potential_elements=["Ni", "Al", "H"]`. It is important to maintain the order of the elements as LAMMPS internally references the elements based on their index, starting from one. The `units` parameter refers to the LAMMPS internal [units](https://docs.lammps.org/units.html) to convert the `ase.atoms.Atoms` object which is defined in Angstrom to the length scale of the LAMMPS simulation. Finally, file_name parameter and the current working directory `working_directory` parameter are designed to select the location where the LAMMPS structure should be written. With the default parameters the LAMMPS structure is written in the `lammps.data` file in the current directory.

Once all the input files are written to the `working_directory` then the LAMMPS calculation can be executed in an external Process using the `subprocess` package:

In [5]:
command = "mpiexec -n 1 --oversubscribe lmp_mpi -in lmp.in"
output = subprocess.check_output(
    command,
    cwd=working_directory,
    shell=True,
    universal_newlines=True,
    env=os.environ.copy(),
)
output.split("\n")

['LAMMPS (27 Jun 2024)',
 'OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)',
 '  using 1 OpenMP thread(s) per MPI task',
 'Reading data file ...',
 '  orthogonal box = (0 0 0) to (4.05 4.05 4.05)',
 '  1 by 1 by 1 MPI processor grid',
 '  reading atoms ...',
 '  4 atoms',
 '  read_data CPU = 0.012 seconds',
 'Reading eam/alloy potential file ../NiAlH_jea.eam.alloy with DATE: 2007-11-30',
 '',
 'CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE',
 '',
 'Your simulation uses code contributions which should be cited:',
 '- Type Label Framework: https://doi.org/10.1021/acs.jpcb.3c08419',
 'The log file lists these citations in BibTeX format.',
 '',
 'CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE-CITE',
 '',
 "WARNING: No fixes with time integration, atoms won't move (src/verlet.cpp:60)",
 'Neighbor list info ...',
 '  update: every = 1 steps, delay = 0 steps, check = yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  m

In addition to the raw output returned by the LAMMPS simulation code, LAMMPS also writes multiple output files. These can be parsed with the `parse_lammps_output_files()` function:

In [6]:
output = parse_lammps_output_files(
    working_directory=working_directory,
    structure=structure,
    potential_elements=potential_elements,
    units="metal",
    dump_h5_file_name="dump.h5",
    dump_out_file_name="dump.out",
    log_lammps_file_name="log.lammps",
)
output

{'generic': {'steps': array([0]),
  'natoms': array([4.]),
  'cells': array([[[4.05000000e+00, 2.47990977e-16, 2.47990977e-16],
          [0.00000000e+00, 4.05000000e+00, 2.47990977e-16],
          [0.00000000e+00, 0.00000000e+00, 4.05000000e+00]]]),
  'indices': array([[0, 0, 0, 0]]),
  'forces': array([[[-5.55111512e-17, -3.39907768e-33, -3.39907768e-33],
          [ 0.00000000e+00,  8.32667268e-17, -5.55111512e-17],
          [ 5.55111512e-17, -5.55111512e-17, -1.66533454e-16],
          [ 0.00000000e+00, -2.22044605e-16,  2.77555756e-17]]]),
  'velocities': array([[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]),
  'unwrapped_positions': array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
          [0.00000000e+00, 2.02500000e+00, 2.02500000e+00],
          [2.02500000e+00, 1.23995488e-16, 2.02500000e+00],
          [2.02500000e+00, 2.02500000e+00, 2.47990977e-16]]]),
  'positions': array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+0

In analogy to the `write_lammps_structure()` function the `working_directory` parameter refers to the directory which contains the output files. The `structure` parameter reefers to the `ase.atoms.Atoms` object which should be used as template to parse the structure from the dump files. This structure is again required as LAMMPS internally references elements only by an index, so the template structure is required to map the elements from the interatomic potential back to the elements of the `ase.atoms.Atoms` object. In the same way the `potential_elements` refers to the list of elements implemented in the specific interatomic potential. The `units` parameter refers to the LAMMPS internal [units](https://docs.lammps.org/units.html) to convert the `ase.atoms.Atoms` object which is defined in Angstrom to the length scale of the LAMMPS simulation. Finally, the parameters `dump_h5_file_name`, `dump_out_file_name` and `log_lammps_file_name` refer to the output file names.